In [86]:
#!pip3 install pandas
#!pip install lxml
#!pip install requests
#!pip install --upgrade certifi
# !pip3 install sqlalchemy
# !pip3 install pymysql

In [88]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import ssl
import re

In [89]:
def getMedicine(max_page):
    df_medi = pd.DataFrame()
    cleanr = re.compile("<.*?>")
    for i in range(1, max_page + 1):
        url = "[url주소]?serviceKey=[서비스키]&pageNo=" + str(i)  
        response = requests.get(url, verify=False).text.encode("utf-8")
        soup = BeautifulSoup(response, 'lxml-xml')    
        items = soup.findAll("item")
   
        for item in items:
            item_name = item.find("itemName").text.replace("\n", "")
            entp_name = item.find("entpName").text.replace("\n", "")
            item_efcy = re.sub(cleanr, "", item.find("efcyQesitm").text).replace("\n", "")
            item_use_method = re.sub(cleanr, "", item.find("useMethodQesitm").text).replace("\n", "")
            item_atpn_warn = re.sub(cleanr, "", item.find("atpnWarnQesitm").text).replace("\n", "")
            item_atpn = re.sub(cleanr, "", item.find("atpnQesitm").text).replace("\n", "")
            item_intrc = re.sub(cleanr, "", item.find("intrcQesitm").text).replace("\n", "")
            item_se = re.sub(cleanr, "", item.find("seQesitm").text).replace("\n", "")
            item_deposit_method = re.sub(cleanr, "", item.find("depositMethodQesitm").text).replace("\n", "")
            item_image = item.find("itemImage").text        

            temp = pd.DataFrame(([[item_name, entp_name, item_efcy, item_use_method, item_atpn_warn, item_atpn, item_intrc, item_se, item_deposit_method, item_image]]), 
                    columns = ["제품명", "업체명", "효능", "사용법", "주의사항 경고", "주의사항", "상호작용", "부작용", "보관법", "낱알이미지"])

            df_medi = pd.concat([df_medi, temp])
            df_medi = df_medi.reset_index(drop=True)
    return df_medi

In [ ]:
# df = getMedicine(441)

In [23]:
from sqlalchemy import create_engine
import pymysql
import sys

In [15]:
# df.to_csv('./medicine_data.csv', index=False)

## 추출한 파일 불러오기

In [77]:
df_new = pd.read_csv('./medicine_data.csv')

In [78]:
df_new = df_new.fillna("없음")

In [79]:
is_null = df_new.isnull().sum()

In [80]:
print(is_null)

item_name              0
entp_name              0
item_efcy              0
item_use_method        0
item_atpn_warn         0
item_atpn              0
item_intrc             0
item_se                0
item_deposit_method    0
item_image             0
dtype: int64


In [87]:
df_new.head(3)

,item_name,entp_name,item_efcy,item_use_method,item_atpn_warn,item_atpn,item_intrc,item_se,item_deposit_method,item_image
0,활명수,동화약품(주),"이 약은 식욕감퇴(식욕부진), 위부팽만감, 소화불량, 과식, 체함, 구역, 구토에 ...","만 15세 이상 및 성인은 1회 1병(75 mL), 만 11세이상~만 15세미만은 ...",없음,만 3개월 미만의 젖먹이는 이 약을 복용하지 마십시오.이 약을 복용하기 전에 만 1...,없음,없음,습기와 빛을 피해 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.,없음
1,신신티눈고(살리실산반창고)(수출명:SINSINCORNPLASTER),신신제약(주),"이 약은 티눈, 못(굳은살), 사마귀에 사용합니다.",이형지로부터 벗겨 이 약제면을 환부(질환 부위)에 대고 테이프로 고정하고 2~5일마...,없음,"이 약에 과민증 환자, 당뇨병, 혈액순환장애 환자는 이 약을 사용하지 마십시오.눈 ...","메토트렉세이트, 설포닐우레아, 다른 국소 적용 약물과 함께 사용 시 의사 또는 약사...","발진, 발적(충혈되어 붉어짐), 홍반(붉은 반점), 가려움, 정상 피부에 닿았을 경...",습기와 빛을 피해 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.,없음
2,아네모정,삼진제약(주),"이 약은 위산과다, 속쓰림, 위부불쾌감, 위부팽만감, 체함, 구역, 구토, 위통, ...","성인 1회 2정, 1일 3회 식간(식사와 식사때 사이) 및 취침시에 복용합니다.",없음,"투석요법을 받고 있는 환자, 수유부, 만 7세 이하의 어린이, 갈락토오스 불내성, ...","위장진통ㆍ진경제, 테트라사이클린계 항생제와 함께 복용하지 마십시오.","발진, 충혈되어 붉어짐, 가려움, 드물게 입이 마르는 증상, 변비 또는 설사 등이 ...",습기와 빛을 피해 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...


In [85]:
db_connection_str = 'mysql+pymysql://[유저명]:[비밀번호]@[주소]:[포트주소]/[데이터베이스이름]?charset=utf8'
db_connection = create_engine(db_connection_str, encoding='utf-8')
conn = db_connection.connect()
df_new.to_sql(name='[데이터베이스명]', con=db_connection, if_exists='append', index=False)

4409